# Compare-Algorithm

In [90]:
import os

def writeToCSV(beeid, indirection, outdirection):
    # write to MaschineLearnedData.csv
    #print("Error: File does not exist!")
    attache = 'a'
    if not (os.path.isfile('./result.csv')):
        attache = 'w'
        with open('result.csv', attache) as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',', quotechar='\"', quoting=csv.QUOTE_MINIMAL)
            filewriter.writerow(["bee_id", "in_direction", "out_direction"])
        attache = 'a'
                    
    with open('result.csv', attache) as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',', quotechar='\'', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow([beeid, indirection, outdirection]) 
        
def formatFileName(filename):
    return filename.split('_')[-1].split('.')[0]

In [111]:
""" filder dublicates from csv """
import pandas as pd

# delete existing result.csv file to prevent errors
if (os.path.isfile('./result.csv')):
    os.remove("result.csv")

# store data in pandas
gtd = pd.read_csv('GroundTruthData.csv')
gtd = pd.DataFrame(data=gtd)

mld = pd.read_csv('MaschineLearnedData.csv')
mld = pd.DataFrame(data=mld)

# delete dublications
print('Delete dublication from input.')
mld = mld.drop_duplicates(['bee_id', 'time_in', 'time_out'], keep='first',)
mld.to_csv('MaschineLearnedData.csv', encoding='utf-8', index=False)

print('Start compare algorithm: ')
for indexMLD, rowMLD in mld.iterrows():
    print('.', end='')
    for indexGTD, rowGTD in gtd.iterrows():
        boolIn, boolOut, boolBoth, boolBothFalse = False, False, False, False
        
        if (rowMLD['time_in'] == rowGTD['timestamp_in'] and rowMLD['bee_id'] == rowGTD['bee_id']):
            boolBothFalse = True
            if (rowMLD['in_direction'] == rowGTD['out_direction']):
                boolIn = True
                      
        if (rowMLD['time_out'] == rowGTD['timestamp_out'] and rowMLD['bee_id'] == rowGTD['bee_id']):
            boolBothFalse = True
            if (rowMLD['out_direction'] == rowGTD['out_direction']):
                boolOut = True
        
        if (boolIn and boolOut):
            boolBoth = True
            writeToCSV(int(rowMLD['bee_id']), "OK", "OK")
        
        if ((boolIn or boolOut) and not boolBoth):
            if (boolIn):
                writeToCSV(int(rowMLD['bee_id']), "OK", "mismatch")
            else:
                writeToCSV(int(rowMLD['bee_id']), "mismatch", "OK")
        
        if ((boolBothFalse == True and boolIn == False) and (boolBothFalse == True and boolOut == False)):
            writeToCSV(int(rowMLD['bee_id']), "mismatch", "mismatch")
    
result = pd.read_csv('result.csv')
result = pd.DataFrame(data=result)

lenResult = len(result)
iterateIn, iterateOut, iterateAll = 0, 0, 0

for index, row in result.iterrows():
    if (row['in_direction'] == "OK" and row['out_direction'] == "OK"):
        iterateAll += 1
    if (row['in_direction'] == "OK"):
        iterateIn += 1
    if (row['out_direction'] == "OK"):
        iterateOut += 1

try:
    print("\n")
    print('For ',len(mld), 'Bees with ', len(result), ' comparable bees: ',round(((len(result)/len(mld))*100),2), 'correct bees found.' )
    print(round(((iterateAll / lenResult) * 100),2), "% total correctness")
    print(round(((iterateIn / lenResult) * 100), 2), "% Ingoing correctness")
    print(round(((iterateOut / lenResult) * 100), 2), "% Outgoing correctness")
    print(round((((iterateAll + iterateIn + iterateOut) / (3*lenResult)) * 100), 2), "% Average correctness")

except ZeroDivisionError:
    print("Nothing to compare!")

print('Done. Please check result.csv file or hit \'result\' for more details.')

Delete dublication from input.
Start compare algorithm: 
....................................................................................................................................................................................................................

For  212 Bees with  96  comparable bees:  45.28 correct bees found.
15.62 % total correctness
32.29 % Ingoing correctness
63.54 % Outgoing correctness
37.15 % Average correctness
Done. Please check result.csv file or hit 'result' for more details.
